In [1]:
import datetime
import netCDF4
import numpy as np
from matplotlib import pyplot as plt
import copy
%matplotlib inline

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import AtrousConvolution2D, Convolution2D, LocallyConnected2D, SpatialDropout2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from theano import tensor as T

Using Theano backend.


In [3]:
varList1 = ['air', 'prmsl', 'shum', 'uwnd', 'vwnd']
varDict1 = {
    'air'   : '../NewFiles/air_scaled_3.2m.2009.nc',
    'prmsl' : '../NewFiles/prmsl_scaled_3.2009.nc',
    'shum'  : '../NewFiles/shum_scaled_3.2m.2009.nc',
    'uwnd'  : '../NewFiles/uwnd_scaled_3.10m.2009.nc',
    'vwnd'  : '../NewFiles/vwnd_scaled_3.10m.2009.nc'
}

In [4]:
varList2 = ['air_anom', 'prmsl_anom', 'shum_anom', 'uwnd', 'vwnd']
varDict2 = {
    'air_anom'   : '../NewFiles/air_30day_scaled_3.2m.2009.nc',
    'prmsl_anom' : '../NewFiles/prmsl_30day_scaled_3.2009.nc',
    'shum_anom'  : '../NewFiles/shum_30day_scaled_3.2m.2009.nc',
    'uwnd'       : '../NewFiles/uwnd_scaled_3.10m.2009.nc',
    'vwnd'       : '../NewFiles/vwnd_scaled_3.10m.2009.nc'
}

In [5]:
varList3 = ['ugrad_air', 'vgrad_air', 'ugrad_prmsl', 'vgrad_prmsl', 'ugrad_shum', 'vgrad_shum', 'uwnd', 'vwnd']
varDict3 = {
    'ugrad_air'   : '../NewFiles/gradient_air_scaled_3.2m.2009.nc',
    'vgrad_air'   : '../NewFiles/gradient_air_scaled_3.2m.2009.nc',
    'ugrad_prmsl' : '../NewFiles/gradient_prmsl_scaled_3.2009.nc',
    'vgrad_prmsl' : '../NewFiles/gradient_prmsl_scaled_3.2009.nc',
    'ugrad_shum'  : '../NewFiles/gradient_shum_scaled_3.2m.2009.nc',
    'vgrad_shum'  : '../NewFiles/gradient_shum_scaled_3.2m.2009.nc',
    'uwnd'        : '../NewFiles/uwnd_scaled_3.10m.2009.nc',
    'vwnd'        : '../NewFiles/vwnd_scaled_3.10m.2009.nc'
}

In [6]:
varList4 = ['ugrad_air_anom', 'vgrad_air_anom', 'ugrad_prmsl_anom', 'vgrad_prmsl_anom', 'ugrad_shum_anom', 'vgrad_shum_anom', 'uwnd', 'vwnd']
varDict4 = {
    'ugrad_air_anom'   : '../NewFiles/gradient_air_30day_scaled_3.2m.2009.nc',
    'vgrad_air_anom'   : '../NewFiles/gradient_air_30day_scaled_3.2m.2009.nc',
    'ugrad_prmsl_anom' : '../NewFiles/gradient_prmsl_30day_scaled_3.2009.nc',
    'vgrad_prmsl_anom' : '../NewFiles/gradient_prmsl_30day_scaled_3.2009.nc',
    'ugrad_shum_anom'  : '../NewFiles/gradient_shum_30day_scaled_3.2m.2009.nc',
    'vgrad_shum_anom'  : '../NewFiles/gradient_shum_30day_scaled_3.2m.2009.nc',
    'uwnd'             : '../NewFiles/uwnd_scaled_3.10m.2009.nc',
    'vwnd'             : '../NewFiles/vwnd_scaled_3.10m.2009.nc'
}

In [7]:
xBounds = [slice(10,80), slice(23, 100)]
yBounds = xBounds
shape = [70, 77]

In [8]:
varDict = varDict2
varList = varList2
numVars = len(varList)

In [9]:
data = netCDF4.Dataset(varDict[varList[0]], 'r')

gt_data = netCDF4.Dataset('../NewFiles/codsus_scaled_3.2009a.nc')

In [10]:
time = data.variables['time']

gt_time = gt_data.variables['time']

ytimes = netCDF4.num2date(gt_time[...], gt_time.units, gt_time.calendar)

xindices = netCDF4.date2index(ytimes, time)

In [11]:
numTimes = len(xindices)

del time
del ytimes
del gt_time
del data

x = np.ma.zeros((numTimes, numVars, shape[0], shape[1]))

In [12]:
for i,v in enumerate(varList):
    data = netCDF4.Dataset(varDict[v], 'r')
    x[:,i,...] = data.variables[v][xindices,xBounds[0],xBounds[1]]
    del data

x = x.transpose((0,2,3,1))

In [13]:
y = gt_data.variables['front_images'][:,:,yBounds[0],yBounds[1]]

y = y.transpose((0,2,3,1))

indices = y.nonzero()

y[indices] = 1

mask = np.ma.getmaskarray(y)

y[mask] = 0

for i in range(0,4):
    indices = np.where(y[...,i] == 1)

    y[indices[0], indices[1], indices[2], i+1:] = 0

In [14]:
print x.shape, y.shape
print y[0,...,0].max()

(2877, 70, 77, 5) (2877, 70, 77, 5)
1


In [15]:
#means = [x_tr[:,v].mean() for v in range(numVars)]

In [16]:
#for v in range(numVars):
#    x_tr[:,v] -= means[v]

In [17]:
#stds = [x_tr[:,v].std() for v in range(numVars)]

In [18]:
#for v in range(numVars):
#    x_tr[:,v] /= stds[v]

In [19]:
mags = [abs(x[...,v]).max() for v in range(numVars)]
for v in range(numVars):
    x[...,v] /= mags[v]

In [20]:
x_mask = x.mask

In [21]:
x = np.asarray(x)

In [22]:
if True == isinstance(x_mask, np.ndarray):
    x[x_mask] = 0

In [23]:
model = Sequential()

num_layers = 3

model.add(ZeroPadding2D(input_shape=x.shape[1:], padding=(2,2), dim_ordering='tf'))

for i in range(num_layers-1):
    model.add(Convolution2D(nb_filter=3*numVars, nb_col=5, nb_row=5, activation='relu', dim_ordering='tf'))
    model.add(ZeroPadding2D(padding=(2,2), dim_ordering='tf'))

#model.add(SpatialDropout2D(0.5, dim_ordering = 'tf'))
model.add(Convolution2D(nb_filter=5, nb_col=5, nb_row=5, activation='sigmoid', dim_ordering='tf'))

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))
model.compile(loss='binary_crossentropy', metrics = ['accuracy'], optimizer=Adam(lr=0.01))

In [ ]:
model.fit(x[0:500,...], y[0:500,...], nb_epoch=100, batch_size=32, validation_split = 0.2, shuffle = True)

Train on 400 samples, validate on 100 samples
Epoch 1/100
400/400 [==============================] - 10s - loss: 0.2053 - acc: 0.9720 - val_loss: 0.1313 - val_acc: 0.9905
Epoch 2/100
400/400 [==============================] - 10s - loss: 0.1114 - acc: 0.9904 - val_loss: 0.1062 - val_acc: 0.9905
Epoch 3/100
400/400 [==============================] - 10s - loss: 0.0910 - acc: 0.9904 - val_loss: 0.0878 - val_acc: 0.9905
Epoch 4/100
288/400 [====================>.........] - ETA: 2s - loss: 0.0759 - acc: 0.9903

In [ ]:
preds = model.predict(x[0:500])

In [ ]:
plt.imshow(preds[0,...,1], cmap='gray')

In [ ]:
plt.imshow(255*y[0,...,1], cmap='gray')

In [ ]:
plt.imshow(x[0,...,0])